## Algorithm for knee excercise tracker

In [1]:
# Imports used
import cv2 
import mediapipe as mp
import PoseMod as pm 
import time 
import numpy as np

# Regularly used parameters
# Font type.
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors in RGB values
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
pink = (255, 0, 255)

# To begin capture from given video or through webcam
cap = cv2.VideoCapture('KneeBendVideo.mp4')

# Reading first frame
ret, img = cap.read()

# Resizing image to improve processing speed
img = cv2.resize(img,(400,400),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)

fps = cap.get(cv2.CAP_PROP_FPS)

# Initializing pose detector from PoseModule
detector = pm.poseDetector()

# Number of reps
count = 0

# Waitkey's interval
interval = 1

# For storing all the frame differences (Difference between previous frame and current frame)
frame_corr = []

# File name for output video
outfilename = 'saiganesh_output_video.avi'

# For storing the frames at each interval 
frames = []

# Number of seconds a person must hold the position
sec = 8

# Duration for flag messages
d = 0

# Threshold for comparing previous and current image
threshold = 10

# Initializing previous frame
prev_frame = None

# Creating the output videowriter
height, width, nchannels = img.shape

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter( outfilename,fourcc, 18, (width,height))

# Function to find the angle between three given points
def find_angle_knees(img, p1 = 23, p2 = 25, p3 = 27):
    # Left Leg
    angle = detector.get_angle(img, p1, p2, p3, draw=True)
    
    return angle

# Finding the time value of last frame
last_t = 0

frame_c = 0

frame_check = False

# Going through the full input file
while True:

    ret, img = cap.read()

    # If image is returned by videocapture
    if(ret==True):
        img = cv2.resize(img,(400,400),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)

    if not ret:
        break
    
    # The below condition is to find the difference between previous and current frame
    # It also tells us the presence of bad or fluctuating frames
    if(prev_frame is not None):

        # Finding difference
        diff = abs(np.sum(np.absolute(img-prev_frame))/np.size(img))
    
        frame_corr.append(diff)

        # cv2.putText(img, str(frame_c), (200,380), font, 1, blue, 2)
        frame_c += 1

        if abs(diff)<threshold:
            # cv2.putText(img, 'WRONG', (70,240), font, 1, blue, 2)
            # cv2.putText(img, str(int(diff)), (240,240), font, 1, blue, 2)
            frame_check = False
        else:
            # cv2.putText(img, 'CORRECT', (50,240), font, 1, blue, 2)
            # cv2.putText(img, str(int(diff)), (240,240), font, 1, blue, 2)
            frame_check = True

    prev_frame = img
    
    # For finding the number of seconds elapsed
    now = time.time()
    dt = now - last_t
    last_t = now

    # Detect the pose and obtain landmarks
    img = detector.get_pose(img, False)
    lmList = detector.get_position(img, False)

    flag_message = ''
    
    # When a pose is detected
    if len(lmList) != 0:
        
        ang = find_angle_knees(img)

        # When angle is lesser than 145 (BENT)
        if(ang<145):
            
            if(sec <= 1):
                flag_message = 'REP COMPLETE!'
            else: flag_message = 'GOOD! KEEP GOING!'

            if(frame_check is False):
                sec = sec
            elif(sec >=1 ) : 
                sec -= dt
            

        # When angle is greater than 145 (NOT BENT)
        if(ang>145):
            if(frame_check is False):
                sec = sec
            if(sec<1):
                count = count + 1
            elif(sec>1 and sec<8):
                # If the person does not bend for 8 seconds
                flag_message = 'KEEP KNEES BENT FOR LONGER'
                d = 15
            elif(ang<150 and sec==8):
                if(d > 0):
                    flag_message = 'KEEP KNEES BENT FOR LONGER'
                    d = d - 1
                else:
                    flag_message = 'BEND KNEES'
            sec = 8


        cv2.rectangle(img, (270,10), (400,40), blue, -1)
        cv2.rectangle(img, (0,94), (120,130), (25,117,16), -1)

        cv2.putText(img, flag_message, (0,80), font, 0.5, dark_blue, 2)
        cv2.putText(img, 'Reps : '+str(int(count)), (280,35), font, 0.8, red, 2)
        cv2.putText(img, 'Time : '+str(int(sec)), (0,120), font, 0.8, pink, 2)

    # Wait for input to quit the while loop
    key = cv2.waitKey(interval) & 0xFF

    # Exit if the key pressed is q
    if key == ord('q'):
        print('Interrupt key q received' )
        break

    cv2.imshow("image", img)
    frames.append(img)

cap.release()

# The below code will remove all the bad frames obtained
bad_locs = []
count_bad = 0
end_val = -1

# Allowing an error rate of two frames
err_frames = 2

temp_f = []

# Going through each frame correlation
for i in range(len(frame_corr)):

    # When it is a bad frame
    if(frame_corr[i]<threshold):
        err_frames = 2
        bad_locs.append(i)
        count_bad = count_bad + 1
    
    # If it is a good frame
    else:
        if(count_bad>=9 and err_frames<1):
            #print(bad_locs)
            temp_f.append(frames[end_val+1:bad_locs[0]-2])
            end_val = bad_locs[-1]
        if(err_frames>0):
            err_frames = err_frames - 1
            continue
        bad_locs = []
        count_bad = 0

    if(i==len(frame_corr)-1 and i-end_val>2):
        temp_f.append(frames[end_val+1:])

# Obtaining the final frames list and writing it to the videowriter
final_f = []
for l in temp_f:
    final_f += l

del temp_f

for i in final_f:
    out.write(i)

# Release the videowriter
out.release()

print('Video processed' )

Video processed
